In [17]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [18]:
import yaml

from featurebyte.tile import TileSnowflake
import snowflake.connector as connector

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
with open('../config.yml', 'r') as file:
    config = yaml.safe_load(file)

snow_config = config['snowflake']
github_config = config['github']

In [20]:
ds = 'tile_ds'

user=snow_config['user']
password=snow_config['password']
url=snow_config['account']
database=snow_config[ds]['database']
schema=snow_config[ds]['schema'] 
warehouse=snow_config[ds]['warehouse']

conn = connector.connect(user=user, password=password, account=url, database=database, schema=schema,
                                      warehouse=warehouse)

# Test 1: Manual Generate Tiles

In [21]:
feature_name = 'feature1'
window_end_minute = 3
blind_spot_seconds = 3
frequency = 5

column_names = ["tile_start_ts", "product_action", "cust_id", "value"]

tile_sql = "select tile_start_ts, product_action, cust_id, value from tile_testing " \
    + " where tile_start_ts >= FB_START_TS " \
    + " and tile_start_ts < FB_END_TS "
    
tile_mgr = TileSnowflake(conn, feature_name, window_end_minute, blind_spot_seconds, frequency, tile_sql, column_names)

In [22]:
tile_mgr.generate_tiles(table_name="FEATURE1_TILE_ONLINE", 
                        start_ts='2022-05-31 23:38:00', 
                        end_ts='2022-06-01 00:00:00')

[I 220530 16:51:43 snowflake_tile:65] 
        call SP_TILE_GENERATE(
            'select tile_start_ts, product_action, cust_id, value from tile_testing  where tile_start_ts >= \'2022-05-31 23:38:00\'  and tile_start_ts < \'2022-06-01 00:00:00\' ', 3, 5, 'tile_start_ts,product_action,cust_id,value', 'FEATURE1_TILE_ONLINE'
        )
    


# Test 2: Online Tile Monitoring

* Manually change Tile values
* Manually run scheduled stored proc
* Monitoring records generated


In [24]:
tile_mgr.schedule_tiles(tile_type="online", end_ts='2022-06-01 00:00:00', monitor_periods=5)

[I 220530 16:54:23 snowflake_tile:89] 
        call SP_TILE_GENERATE_SCHEDULE(
            'FEATURE1', 3, 3, 5, 'select tile_start_ts, product_action, cust_id, value from tile_testing  where tile_start_ts >= FB_START_TS  and tile_start_ts < FB_END_TS ',
            'tile_start_ts,product_action,cust_id,value', 'ONLINE', 5, '2022-06-01 00:00:00'
        )
    


# Test 3: Create Schedule Task for Ongoing Online Tile Generation and Monitoring

In [8]:
tile_mgr.schedule_tiles(tile_type="online")

[I 220527 22:44:14 snowflake_tile:89] 
        CREATE OR REPLACE TASK TASK_FEATURE1_ONLINE_TILE
          WAREHOUSE = COMPUTE_WH
          SCHEDULE = 'USING CRON 3-59/5 * * * * UTC'
        AS
            call SP_TILE_GENERATE_SCHEDULE(
                'FEATURE1', 3, 3, 5, 'select tile_start_ts, product_action, cust_id, value from tile_testing  where tile_start_ts >= FB_START_TS  and tile_start_ts < FB_END_TS ',
                'tile_start_ts,product_action,cust_id,value', 'ONLINE', 2, NULL
            )
    
[I 220527 22:44:14 snowflake_tile:93] task_name: TASK_FEATURE1_ONLINE_TILE
